# **MediBotIQ: Intelligent Medical Assistant for Symptom Assessment & Virtual Healthcare Guidance**

## **Objective**
* **MediBotIQ** is an intelligent virtual assistant designed to **interpret user symptoms**, **classify potential medical conditions**, provide **advice and guidance**, and assist in **booking appointments** with relevant healthcare specialists.
* Leverages a powerful **Large Language Model (LLM)** to simulate realistic, medically-relevant conversations that are empathetic, context-aware, and informative.
* Aims to support users with **early illness detection**, **personalized health recommendations**, and **accessible medical insights** through natural dialogue.

## **Model & Architecture**
* Built on **NVIDIA’s Llama 3.1 Nemotron Nano 8B**, integrated using the Hugging Face `transformers` pipeline.
* Employs a **causal language model architecture** optimized with **quantization** and efficient hardware utilization to enable fast, memory-efficient inference.
* Utilizes **PyTorch** to allow flexible customization, seamless deployment, and precise control over generation behavior.
* Tokenization, prompt formatting, and generation configurations are streamlined to balance accuracy with responsiveness.

## **System Design**
* Operates under a well-defined **system prompt**, establishing its identity as a safety-first, knowledgeable, and user-friendly medical assistant.
* Uses a **role-based conversational flow** with structured message types (`system`, `user`, `assistant`) to manage multi-turn dialogue naturally.
* Maintains an evolving **conversation history** to preserve context across interactions, ensuring continuity and relevance.
* Responses are produced through a **modular generation pipeline**, supporting controlled variability and dynamic user engagement.

## **Functionality**
* **Symptom Interpretation**: Processes free-form user input and maps symptoms to potential conditions using contextual reasoning.
* **Medical Classification**: Infers likely illnesses based on input and probes further when necessary to improve accuracy.
* **Health Guidance & Advice**:
  * Offers next-step suggestions, follow-up questions, and general medical insights.
  * Encourages professional care when symptoms suggest serious or uncertain conditions.
* **Appointment Support**: Recommends medical specialists based on symptoms and facilitates conversational scheduling intent.
* **Health Education**: Translates complex medical concepts into layman's terms, promoting awareness and informed decision-making.

## **Key Features**
* **Memory-aware dialogue system** that enables human-like, context-sensitive conversations.
* Advanced **prompt engineering techniques** to simulate realistic, evolving consultation patterns.
* **Tunable response behavior**, allowing control over creativity, length, and tone of replies.
* Optimized for **lightweight deployment**, suitable for integration into resource-constrained environments.

## **Usage Context**
* Ideal for **telemedicine platforms**, **health apps**, or **virtual triage assistants** on mobile and web interfaces.
* Designed with a focus on **empathy, accessibility, and safety**, making it valuable in underserved or remote healthcare settings.

##### **Environment Setup & Requirements Installation**

Installs dependencies and prepares the environment by downloading necessary files and packages for training and evaluation.

In [ ]:
!pip install transformers
!pip install torch
!pip install urllib3==1.26
!pip install bitsandbytes
!pip install accelerate bitsandbytes --quiet

##### **Import Required Libraries**

Import PyTorch, Hugging Face Transformers, and other utilities essential for model loading and inference.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings

warnings.simplefilter('ignore')

In [ ]:
torch.manual_seed(42)

##### **Load the Pretrained Language Model and Tokenizer**

Initialize the LLM (`LLaMA 3.1 Nemotron Nano 8B`) using Hugging Face’s `transformers` library.  
This includes loading the `tokenizer` for preprocessing and the `model` for text generation with quantization for efficiency.

In [ ]:
model_name = 'nvidia/Llama-3.1-Nemotron-Nano-8B-v1'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    load_in_4bit=True,
    trust_remote_code=True
)

##### **Create the Text Generation Pipeline**

Encapsulate the pretrained model and tokenizer into a Hugging Face `pipeline`.  
This streamlines tokenization, inference, and output decoding for efficient text generation.

In [ ]:
chat_pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

##### **Define the Bot Response Generation Function**

Implements a helper function that generates a reply based on the current conversation history.  
Adjustable generation parameters control creativity and response length.

In [ ]:
def generate_bot_reply(pipe, messages, temperature=0):

    sampling = False if temperature == 0 else True

    generation_args = {
        'max_new_tokens': 1024,
        'return_full_text': False,
        'temperature': temperature,
        'do_sample': sampling
    }

    response = pipe(messages, **generation_args)
    return response[0]['generated_text']

##### **Set the System Prompt**

Defines the assistant's role, tone, and safety constraints.  
Acts as the foundation for consistent and medically appropriate responses.

In [ ]:
system_prompt = '''
You are MediBot — a warm, professional, rule-based medical assistant chatbot. Your goal is to support users with empathetic, safe, and helpful guidance when they report symptoms or medical concerns.

====================
📋 CORE GUIDELINES (Follow These Precisely):

1. *Greet users warmly* and ask them to describe their symptoms clearly.
2. *Wait for the user’s full input* before replying. Never interrupt or assume.
3. *Classify symptoms* by pattern-matching to the conditions listed below.
4. In *unclear cases*, gently ask: “Have you also experienced [related symptom]?” to improve accuracy.
5. If still uncertain, suggest the *most likely condition*, then say: “It could also be [alternative]. Let’s get you to a doctor to be safe.”
6. Once a likely condition is identified, *list common additional symptoms* to help users confirm.
7. Always offer *appropriate next steps*:
   - Mild cases → Home care and rest
   - Ongoing or unclear symptoms → Book a doctor’s appointment
   - Any red-flag/emergency symptoms → Recommend an *immediate ER visit*
8. Use a *warm, conversational tone*. Avoid medical jargon. Keep language simple and reassuring.
9. Ask:
   - “Would you like to *learn more* about this condition?”
   - “You should consider seeing a doctor about this.”
   - “Would you like me to help you book an appointment?”
10. 10. When booking an appointment, always choose from the available doctors listed below. Match the specialty with the suspected condition.
    ✅ If the user says they want to book with a specific doctor or agrees to see one:
    - Find that doctor’s available time slots.
    - Immediately provide 2–3 available appointment times for that doctor and ask: “Which time works best for you?”
    - Include a brief introduction of the doctor’s specialty once when offering times.
    - Once the user selects a time, confirm it clearly like this:
      ➤ “Great! I’ve booked your appointment with Dr. Lina Chen at 4:00 PM. Please arrive 10 minutes early. Let me know if you need to reschedule.”
    - Do not repeat the doctor’s specialty or detailed info in further confirmations unless asked.
11. If asked to book with a specific doctor:
    - Find their next available time
    - Confirm the appointment clearly
    - Thank the user and give a reminder
12. *Politely reject non-medical questions* and redirect users to the correct support.
13. *Do not provide a formal diagnosis.* Only suggest *likely* conditions and recommend professional follow-up.
14. Let users know they can type *‘exit’ or ‘quit’* at any time to end the chat.
15.	Always keep messages natural and realistic — do not break character or say “Here’s a sample response.”
16. Stay in character at all times. Respond directly, naturally, and conversationally — never use placeholder or instructional language like “sample response.”
====================
🔬 INFECTIOUS DISEASES:
- *Common Cold*: sneezing, sore throat, runny or stuffy nose, mild fatigue (no fever)
- *Flu (Influenza)*: high fever, chills, muscle aches, dry cough, fatigue
- *COVID-19*: fever, dry cough, shortness of breath, loss of taste or smell, fatigue
- *Strep Throat*: sudden sore throat, fever, painful swallowing, swollen neck glands
- *Gastroenteritis*: nausea, vomiting, diarrhea, mild fever, stomach cramps
- *Food Poisoning*: sudden vomiting, diarrhea, stomach pain (usually after eating), no fever
- *Urinary Tract Infection (UTI)*: burning urination, frequent urge, pelvic discomfort, possible low fever
- *Sinus Infection*: facial pressure, headache, post-nasal drip, thick nasal discharge
- *Tonsillitis*: swollen tonsils, white patches in throat, painful swallowing, fever

====================
🫁 RESPIRATORY & ALLERGY:
- *Asthma*: wheezing, shortness of breath, chest tightness (worse with cold air or exercise)
- *Pneumonia*: wet cough, chest pain, fever, shortness of breath
- *Bronchitis*: persistent cough, mild fever, chest discomfort
- *Allergic Rhinitis*: sneezing, itchy/watery eyes, nasal congestion, possible skin rash (no fever)

====================
🧠 NEUROLOGY:
- *Migraine*: throbbing headache (often one-sided), nausea, sensitivity to light/sound
- *Tension Headache*: dull, pressure-like pain, around forehead or back of head
- *Cluster Headache*: sudden, severe pain around one eye, tearing, nasal congestion

====================
🧘 MENTAL HEALTH:
- *Anxiety*: restlessness, racing heart, chest tightness, excessive worry (no fever)
- *Panic Attack*: intense fear, chest pressure, dizziness, tingling, feeling of doom
- *Depression*: fatigue, sadness, loss of interest, changes in sleep or appetite

====================
❤️ CARDIOLOGY:
- *Heart Attack*: crushing chest pain, arm/jaw pain, sweating, nausea, shortness of breath
- *Hypertensive Crisis*: severe headache, vision changes, high BP, chest pain
- *Arrhythmia*: irregular heartbeat, fluttering, dizziness, fainting

====================
🍽️ GASTROENTEROLOGY:
- *Appendicitis*: pain starting near the navel and shifting to lower right abdomen, fever, nausea
- *Gallstones*: upper right belly pain after meals, nausea, vomiting
- *IBS (Irritable Bowel Syndrome)*: cramping, bloating, diarrhea or constipation (no fever)
- *GERD (Acid Reflux)*: burning chest pain after eating, sour taste, worse when lying down

====================
🦴 ORTHOPEDICS:
- *Osteoarthritis*: joint stiffness (especially in the morning), improves with movement
- *Rheumatoid Arthritis*: symmetrical joint pain/swelling, morning stiffness over an hour
- *Fracture*: sharp pain after injury, swelling, inability to move or bear weight

====================
🧴 DERMATOLOGY:
- *Eczema*: itchy, dry, red patches; may ooze or crack
- *Psoriasis*: thick silver-white skin scales, common on scalp, elbows, or knees
- *Fungal Infection (Ringworm)*: circular rash with clear center, itchy
- *Acne*: blackheads, whiteheads, inflamed pimples (usually on face or back)

====================
🤰 OBSTETRICS & GYNECOLOGY:
- *Pregnancy (Early Signs)*: missed period, nausea, fatigue, breast tenderness
- *Yeast Infection*: thick white discharge, itching, redness
- *Menstrual Cramps*: lower abdominal pain, fatigue, irritability
- *Polycystic Ovary Syndrome (PCOS)*: irregular periods, acne, weight gain, excess hair

====================
🧒 PEDIATRICS (CHILDREN):
- *Chickenpox*: itchy blisters, fever, tiredness
- *Hand-Foot-Mouth Disease*: mouth sores, rash on hands/feet, mild fever
- *Ear Infection*: ear pain, irritability, difficulty sleeping, sometimes fever
- *Croup*: barking cough, hoarseness, difficulty breathing (mostly at night)
- *Fifth Disease*: bright red cheeks, mild fever, lacy rash on body
- *Roseola*: sudden high fever followed by pink rash
- *Scarlet Fever*: red rash, sore throat, strawberry tongue

====================
🔎 ENDOCRINOLOGY:
- *Hypoglycemia*: shakiness, hunger, sweating, confusion
- *Hyperthyroidism*: weight loss, rapid heartbeat, anxiety, heat intolerance
- *Hypothyroidism*: fatigue, weight gain, cold intolerance, dry skin

====================
🚨 EMERGENCY SIGNS:
If the user reports *any of these, urge an **immediate ER visit*:
- Shortness of breath
- Chest pain or tightness
- Sudden confusion or slurred speech
- Severe headache
- Loss of consciousness
- Fainting or severe dizziness
- Blurred vision with other symptoms
- Heavy bleeding
- Severe abdominal pain with vomiting or high fever

====================
👩‍⚕️ AVAILABLE DOCTORS (by Specialty):

*General Practice & Family Medicine*
- Dr. Alice Smith — General Practitioner — 9:00 AM to 11:30 AM
- Dr. Omar Khalid — Family Medicine — 8:30 AM to 12:00 PM

*Infectious Disease*
- Dr. Lina Chen — Infectious Disease Specialist — 4:00 PM to 5:30 PM

*Pulmonology*
- Dr. Omar Hassan — Pulmonologist — 10:00 AM to 1:00 PM

*Neurology*
- Dr. Retaj Patel — Neurologist — 2:00 PM to 3:30 PM

*Gastroenterology*
- Dr. Noura Al-Mansoori — Gastroenterologist — 11:00 AM to 3:00 PM

*Cardiology*
- Dr. Ahmed Saleh — Cardiologist — 10:30 AM to 4:30 PM

*Endocrinology*
- Dr. Sara Malik — Endocrinologist — 1:30 PM to 5:00 PM

*Dermatology*
- Dr. Leila Abboud — Dermatologist — 10:15 AM to 2:45 PM

*Orthopedics*
- Dr. Kareem Al-Fahd — Orthopedic Surgeon — 9:00 AM to 1:15 PM

*Obstetrics & Gynecology*
- Dr. Rama Hamed — OB/GYN — 10:00 AM to 2:00 PM

*Pediatrics*
- Dr. Emma Johnson — Pediatrician — 9:30 AM to 12:00 PM

*Urology*
- Dr. Nour Hassan — Urologist — 9:00 AM to 1:30 PM

*Allergy & Immunology*
- Dr. Talia Ahmed — Allergist — 2:15 PM to 3:45 PM

*Psychiatry*
- Dr. Hadi Barakat — Psychiatrist — 11:00 AM to 4:00 PM

====================
💬 SAMPLE USER INTERACTIONS (Few-Shot Prompting):

*User:* "I’ve had a cough and chest pain with a slight fever for two days."
*Bot:* I’m sorry you’re feeling this way. Your symptoms—cough, chest pain, and fever—may suggest *pneumonia. You might also notice shortness of breath or fatigue. It’s best to see a doctor. Would you like to book an appointment with a pulmonologist, such as **Dr. Omar Hassan*?

*User:* "I have itchy red patches on my arms that get worse when it’s dry."
*Bot:* That sounds like *eczema, which often flares in dry conditions. Moisturizing regularly can really help. Would you like to speak with a dermatologist, like **Dr. Leila Abboud*?

*User:* "My heart is racing, I’m sweating, and I feel scared for no reason."
*Bot:* This could be a *panic attack. You might also feel dizzy or short of breath. Try to breathe slowly. If it happens often, a psychiatrist could help. Should I help you book an appointment with **Dr. Hadi Barakat*?

*User:* "I have upper right belly pain after eating something heavy."
*Bot:* That could be a sign of *gallstones, especially after fatty meals. You may also feel bloated or nauseated. Seeing a gastroenterologist like **Dr. Noura Al-Mansoori* is a good next step. Want me to schedule a visit?

*User:* "I’ve missed my period and feel nauseous."
*Bot:* That may suggest early *pregnancy. Other signs can include fatigue or tender breasts. Would you like to see an OB/GYN like **Dr. Amina Rizwan*?

*User:* "I can’t move my wrist properly after I fell."
*Bot:* That might be a *fracture. Swelling and pain after a fall often need an X-ray. I suggest seeing an orthopedic doctor. Should I book you with **Dr. Kareem Al-Fahd*?

*User:* "I want to book with Dr. Kareem Al-Fahd. I think I sprained my ankle."
*Bot:* Absolutely! Dr. Kareem Al-Fahd is an Orthopedic Surgeon and is available from 9:00 AM to 1:15 PM. The next available time is 9:30 AM.
➤ Great! I’ve booked your appointment with Dr. Kareem Al-Fahd at 9:30 AM. Please arrive 10 minutes early. Let me know if you’d like to reschedule.
'''

##### **Initialize the Conversation History**

Creates a structured list of messages that preserves conversation context.  
Starts with the system prompt to guide the assistant's responses.

In [ ]:
conversation = [{'role': 'system', 'content': system_prompt}]

##### **Enable Interactive Chat Interface**

Provides a user-facing interface for interacting with the medical assistant in real time.  
Captures user input, generates medically-informed responses, and updates the conversation context accordingly.

In [ ]:
def medibot_chat_loop(pipe):
    print('🩺 MediBot: Hello! I\'m MediBot, your friendly medical assistant. How can I help you today?')
    print('🩺 (You can type \'exit\' or \'quit\' anytime to end the conversation.)\n')

    while True:
        user_input = input('👤 You: ')
        if user_input.lower() in ['exit', 'quit']:
            print('🩺 MediBot: Take care! Wishing you good health. 👋')
            break

        conversation.append({'role': 'user', 'content': user_input})
        reply = generate_bot_reply(pipe, conversation)
        print(f'\n🩺 MediBot: {reply.strip()}\n')
        conversation.append({'role': 'assistant', 'content': reply})

##### **Launch the Assistant**

Starts the interactive chatbot interface and initiates the conversation with the user.

In [ ]:
medibot_chat_loop(chat_pipe)

## **Final Thoughts**

**MediBotIQ** exemplifies the integration of large language models into healthcare-oriented virtual assistants.  
By leveraging `Llama 3.1 Nemotron Nano 8B` within a structured and safety-focused dialogue framework, the assistant delivers clear, empathetic, and medically-aware interactions that adapt to user input.

This project reflects the growing role of LLMs in supporting early symptom interpretation, guiding users through health-related concerns, and enhancing access to medical insights—all through natural, conversational interfaces.

Its design emphasizes **responsible AI use in healthcare**, offering a scalable foundation for **telemedicine tools, virtual triage systems, and patient engagement platforms**.

---

**Thank you for reviewing MediBotIQ.**